<a href="https://colab.research.google.com/github/yxiaoGit/AngularWeb/blob/main/Cal_R_Talk_Robots_Kinetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forward Kinematics

In [1]:
import sys

# Only run interactive demo in Google Colab
if 'google.colab' not in sys.modules:
    print("⚠️ This interactive demo uses ipywidgets and matplotlib. Please copy and run this code in a Google Colab notebook for live GUI.")
else:
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import interactive, FloatSlider, ToggleButtons
    from IPython.display import display

    # Link lengths
    L1, L2, L3 = 1.0, 1.0, 1.0

    def fk(theta1_deg, theta2_deg, theta3_deg):
        """Forward kinematics: plot 3-DOF planar arm given joint angles in degrees."""
        # convert to radians
        θ1 = np.deg2rad(theta1_deg)
        θ2 = np.deg2rad(theta2_deg)
        θ3 = np.deg2rad(theta3_deg)

        # compute joint positions
        x0, y0 = 0, 0
        x1, y1 = L1 * np.cos(θ1), L1 * np.sin(θ1)
        x2, y2 = x1 + L2 * np.cos(θ1 + θ2), y1 + L2 * np.sin(θ1 + θ2)
        x3, y3 = x2 + L3 * np.cos(θ1 + θ2 + θ3), y2 + L3 * np.sin(θ1 + θ2 + θ3)

        # plot
        plt.figure(figsize=(5,5))
        plt.plot([x0, x1, x2, x3], [y0, y1, y2, y3], 'o-', linewidth=3)
        plt.xlim(-3, 3)
        plt.ylim(-3, 3)
        plt.axhline(0, color='gray', linewidth=1)
        plt.axvline(0, color='gray', linewidth=1)
        plt.gca().set_aspect('equal', 'box')
        plt.title(f"Kinematics\nθ₁={theta1_deg:.1f}°, θ₂={theta2_deg:.1f}°, θ₃={theta3_deg:.1f}°")
        plt.text(x3 + 0.1, y3 + 0.1, f"({x3:.2f}, {y3:.2f})", fontsize=12, color='blue')
        plt.show()

    def ik(px, py, phi_deg, elbow):
        """Inverse kinematics: given end-effector pose (px,py,φ°), solve for joint angles."""
        # convert orientation to radians
        φ = np.deg2rad(phi_deg)

        # wrist position
        xw = px - L3 * np.cos(φ)
        yw = py - L3 * np.sin(φ)

        D = (xw**2 + yw**2 - L1**2 - L2**2) / (2 * L1 * L2)
        if abs(D) > 1:
            plt.figure(figsize=(5,5))
            plt.text(0.5, 0.5, 'Target unreachable', ha='center', va='center', fontsize=14)
            plt.axis('off')
            plt.show()
            return

        # analytic solutions for elbow-up/down
        θ2 = np.arctan2(elbow * np.sqrt(1 - D**2), D)
        θ1 = np.arctan2(yw, xw) - np.arctan2(L2 * np.sin(θ2), L1 + L2 * np.cos(θ2))
        θ3 = φ - (θ1 + θ2)

        # convert back to degrees for display
        t1_deg = np.rad2deg(θ1)
        t2_deg = np.rad2deg(θ2)
        t3_deg = np.rad2deg(θ3)

        # reuse fk to plot (it handles the annotation)
        fk(t1_deg, t2_deg, t3_deg)

    # Widgets in degrees
    fk_sliders = {
        'theta1_deg': FloatSlider(value=0, min=-180, max=180, step=1, description='θ₁ (deg)'),
        'theta2_deg': FloatSlider(value=0, min=-180, max=180, step=1, description='θ₂ (deg)'),
        'theta3_deg': FloatSlider(value=0, min=-180, max=180, step=1, description='θ₃ (deg)'),
    }

    ik_sliders = {
        'px': FloatSlider(value=1.0, min=-3.0, max=3.0, step=0.01, description='px'),
        'py': FloatSlider(value=1.0, min=-3.0, max=3.0, step=0.01, description='py'),
        'phi_deg': FloatSlider(value=0, min=-180, max=180, step=1, description='φ (deg)'),
        'elbow': ToggleButtons(options=[('Up', 1), ('Down', -1)], description='Elbow'),
    }

    # Display
    print("🔄 Forward Kinematics Interactive (degrees)")
    fk_widget = interactive(fk, **fk_sliders)
    display(fk_widget)

🔄 Forward Kinematics Interactive (degrees)


interactive(children=(FloatSlider(value=0.0, description='θ₁ (deg)', max=180.0, min=-180.0, step=1.0), FloatSl…

# Inverse Kinematics

In [2]:
    print("\n🎯 Inverse Kinematics Interactive")
    ik_widget = interactive(ik, **ik_sliders)
    display(ik_widget)


🎯 Inverse Kinematics Interactive


interactive(children=(FloatSlider(value=1.0, description='px', max=3.0, min=-3.0, step=0.01), FloatSlider(valu…

# 4DOF Robotic Arm, Redundancy

In [3]:
import sys

# Only run this interactive demo in Google Colab
if 'google.colab' not in sys.modules:
    print("⚠️ This interactive demo uses ipywidgets and matplotlib. Please copy and run this code in a Google Colab notebook.")
else:
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import interactive, FloatSlider, ToggleButtons
    from IPython.display import display

    # Four equal link lengths for the 4‐DOF planar arm
    L1 = L2 = L3 = L4 = 1.0

    def ik4(px, py, phi_deg, elbow, theta4_deg):
        """
        Inverse kinematics for a redundant 4-DOF planar arm:
         - (px,py,phi_deg) is the desired end‐effector pose.
         - 'elbow' chooses the elbow-up/down branch.
         - theta4_deg is the free (redundant) joint angle.
        """
        # Convert orientations to radians
        phi = np.deg2rad(phi_deg)
        theta4 = np.deg2rad(theta4_deg)

        # Compute the wrist‐position for link 4 (back off L4 along phi)
        x3 = px - L4 * np.cos(phi)
        y3 = py - L4 * np.sin(phi)

        # Effective end‐effector orientation seen by the first 3 links
        phi_eff = phi - theta4

        # Compute the wrist for link 3 (back off L3 along phi_eff)
        xw = x3 - L3 * np.cos(phi_eff)
        yw = y3 - L3 * np.sin(phi_eff)

        # Law of Cosines for the "elbow" link of the first 3 links
        D = (xw**2 + yw**2 - L1**2 - L2**2) / (2 * L1 * L2)
        if abs(D) > 1:
            # target not reachable
            plt.figure(figsize=(5,5))
            plt.text(0.5, 0.5, 'Unreachable', ha='center', va='center', fontsize=14, color='red')
            plt.axis('off')
            plt.show()
            return

        # Two solutions for link 2 (elbow)
        theta2 = np.arctan2(elbow * np.sqrt(1 - D**2), D)
        # Shoulder joint angle from geometry
        theta1 = np.arctan2(yw, xw) - np.arctan2(L2 * np.sin(theta2), L1 + L2 * np.cos(theta2))
        # Third joint to match the effective orientation
        theta3 = phi_eff - (theta1 + theta2)

        # Now build the full 4‐DOF chain positions
        p0 = (0.0, 0.0)
        p1 = (L1*np.cos(theta1), L1*np.sin(theta1))
        p2 = (p1[0] + L2*np.cos(theta1 + theta2), p1[1] + L2*np.sin(theta1 + theta2))
        p3 = (p2[0] + L3*np.cos(theta1 + theta2 + theta3), p2[1] + L3*np.sin(theta1 + theta2 + theta3))
        p4 = (p3[0] + L4*np.cos(phi), p3[1] + L4*np.sin(phi))

        # Plot the arm
        plt.figure(figsize=(5,5))
        xs = [p0[0], p1[0], p2[0], p3[0], p4[0]]
        ys = [p0[1], p1[1], p2[1], p3[1], p4[1]]
        plt.plot(xs, ys, 'o-', linewidth=3)
        # Overlay the target
        plt.plot(px, py, 'r*', markersize=12)
        plt.xlim(-4, 4)
        plt.ylim(-4, 4)
        plt.axhline(0, color='gray', linewidth=1)
        plt.axvline(0, color='gray', linewidth=1)
        plt.gca().set_aspect('equal', 'box')
        plt.title(
            f"4-DOF IK\nθ₁={np.degrees(theta1):.1f}°, θ₂={np.degrees(theta2):.1f}°, θ₃={np.degrees(theta3):.1f}°, θ₄={theta4_deg:.1f}°"
        )
        plt.show()

    # Widgets
    ik4_widget = interactive(
        ik4,
        px=FloatSlider(value=1.0, min=-3.0, max=3.0, step=0.01, description='px'),
        py=FloatSlider(value=1.0, min=-3.0, max=3.0, step=0.01, description='py'),
        phi_deg=FloatSlider(value=0, min=-180, max=180, step=1, description='φ (deg)'),
        elbow=ToggleButtons(options=[('Down', 1), ('Up', -1)], description='Elbow'),
        theta4_deg=FloatSlider(value=0, min=-180, max=180, step=1, description='θ₄ (deg)')
    )

    print("🎯 Redundant 4-DOF Inverse Kinematics Demo")
    display(ik4_widget)

🎯 Redundant 4-DOF Inverse Kinematics Demo


interactive(children=(FloatSlider(value=1.0, description='px', max=3.0, min=-3.0, step=0.01), FloatSlider(valu…